Q1a:

MySql Statement:

select e1.name as EmployeeName <br/>
from Employee as e1, Employee as e2 <br/>
where e1.manager_id = e2.id AND e1.Salary > e2.Salary <br/>
<br/>

Explanation: 

Selecting the name will output the employee's name and selecting from 2 tables will get the Cartesian Product of these 2 tables. The 2 conditions in the where clause is used to filter for the names of employees whose salaries are greater than their immediate manager's. 

Q1b:

To find out employees who do not manage anyone, filter out the NULL values in the manager_id col first and check the values in the manager_id col against the values in the id col. For the values in the manager_id col that appeared in the id col, it means that the employee with that specific id manages someone. For the remaining id values, we can sum up their corresponding salary and divide by the count to get the average salary.

Q2:

In [13]:
def exists(v):
    try:
        v
    except NameError:
        return False # v is not defined since there is NameError
    else:
        return True # v is defined


Q3:

In [9]:
import math
def pascalLayer(n):
    
    def binomialCoeff(line, i): # each entry in a layer is a value of a Binomial Coefficient which can be computed using combinations formula
        return math.comb(line, i) # line represents the layer and i represents the entry position in the layer
    
    for line in range(0, n):  # iterate through each layer and print the entries in each layer
        for i in range(0, line + 1): # number of values in each layer is equal to the layer number
            print(binomialCoeff(line, i), " ", end = "")
        print()

pascalLayer(4)

1  
1  1  
1  2  1  
1  3  3  1  
